In [ ]:
import docplex

from docplex.mp.model import Model
import pandas as pd
import numpy as np
import random
from sklearn.neighbors import DistanceMetric
from math import radians

### Distance calculation

In [ ]:
## import of longitudinal and latitudinal data for municipalities in Germany. 
## afterwards: distance-calculation for all flows that are possible
## at the end: randomly chose 600 locations for workshops once, exported it to excel file, that will be used for the profit-calculation

In [ ]:
DF = pd.read_excel("ExcelImport/Masterarbeit_Input_2.xlsx", sheet_name="Locations")

In [ ]:
DF['Breite'] = np.radians(DF['Breite'])
DF['Länge'] = np.radians(DF['Länge'])

In [ ]:
dist = DistanceMetric.get_metric('haversine')

In [ ]:
 dist.pairwise(DF[['Breite','Länge']].to_numpy())*6373

In [ ]:
DF_2=pd.DataFrame(dist.pairwise(DF[['Breite','Länge']].to_numpy())*6373,  columns=DF.ORT, index=DF.ORT)
DF_2

Dataframe distances

In [ ]:
## Distances between all nodes that are necessary
##
##Change manually if want to change the model setting
##

DF_3=DF.iloc[:,[0]]

OEM= "ULM"
REM1= "KASSEL"
REM2 ="ULM"
REM3 ="ESSEN"
REM4 ="HAMBURG"
INT= "NÜRNBERG"
COLL1= "MANNHEIM"
COLL2="HANNOVER"
COLL3="BERLIN (WEST)"
COLL4="KIEL"
REX ="LEIPZIG"


DF_3["dmi"]=DF_2[OEM][INT]
DF_3["diw"]=DF_2[INT].to_numpy()
DF_3["dwc"]=DF_2[COLL1].to_numpy()
DF_3["dwc2"]=DF_2[COLL2].to_numpy()
DF_3["dwc3"]=DF_2[COLL3].to_numpy()
DF_3["dwc4"]=DF_2[COLL4].to_numpy()
DF_3["dwd"]=DF_2[REX].to_numpy()
DF_3["dcd"]=DF_2[COLL1][REX]
DF_3["dcm"]=DF_2[COLL1][OEM]

DF_3["dcr1"]=DF_2[COLL1][REM1]
DF_3["dcr2"]=DF_2[COLL1][REM2]
DF_3["dcr3"]=DF_2[COLL1][REM3]
DF_3["dcr4"]=DF_2[COLL1][REM4]

DF_3["dc2d"]=DF_2[COLL2][REX]
DF_3["dc2m"]=DF_2[COLL2][OEM]

DF_3["dc2r1"]=DF_2[COLL2][REM1]
DF_3["dc2r2"]=DF_2[COLL2][REM2]
DF_3["dc2r3"]=DF_2[COLL2][REM3]
DF_3["dc2r4"]=DF_2[COLL2][REM4]

DF_3["dc3d"]=DF_2[COLL3][REX]
DF_3["dc3m"]=DF_2[COLL3][OEM]

DF_3["dc3r1"]=DF_2[COLL3][REM1]
DF_3["dc3r2"]=DF_2[COLL3][REM2]
DF_3["dc3r3"]=DF_2[COLL3][REM3]
DF_3["dc3r4"]=DF_2[COLL3][REM4]

DF_3["dc4d"]=DF_2[COLL4][REX]
DF_3["dc4m"]=DF_2[COLL4][OEM]

DF_3["dc4r1"]=DF_2[COLL4][REM1]
DF_3["dc4r2"]=DF_2[COLL4][REM2]
DF_3["dc4r3"]=DF_2[COLL4][REM3]
DF_3["dc4r4"]=DF_2[COLL4][REM4]

DF_3["dr1i"]=DF_2[REM1][INT]
DF_3["dr2i"]=DF_2[REM2][INT]
DF_3["dr3i"]=DF_2[REM3][INT]
DF_3["dr4i"]=DF_2[REM4][INT]



### Define model

In [ ]:
model = Model (name = "Baseline model")

### Definition of sets

In [ ]:
d= [1] #different reprocessing option
c= [1,2,3,4] #sets of collection center
m= [1] #OEM manufacturing
i=[1] #intermediate collection center
t=range (1,61) #time periods in months
w= range(1,601) #workshops
s=[1,2,3] #sets of scenarios for yield of returning batteries (beta) (1=pessimistic, 2=medium, 3=optimistic)

### Definition of parameters 

In [ ]:
data = pd.ExcelFile("ExcelImport/Masterarbeit_Input_2.xlsx")


##Fixed cost
#Fixed cost opening intermediate collection center
Fi = data.parse("fixed cost", header=None)[1][0]
#Fixed cost opening remanufacturing facility per month (depreciation)
Fr = data.parse("fixed cost", header=None)[1][1]
#Fixed cost opening collection center for used batteries per month (depreciation)
Fc = data.parse("fixed cost", header=None)[1][2]

##Yields
#rate of return of batteries
alpha = np.tile(np.array(data.parse("yields", header=None)[1][0]), 601,)
#rate of severly damaged batteries going directly and alone to ReX (scenarios)
beta = np.tile([0.1, 0.04, 0.01], (601,61,1))
#probability for each scenario (1= pessimistic, 2=normal, 3=optimisitic)
prob = [0.15,0.6,0.25]
#rate of batteries going to reman instead of Re-X.
delta= np.tile(np.array(data.parse("yields", header=None)[1][2]), 601,)

##Capacities
#Collection point used battery
capc= data.parse("capa", header=None)[1][0]
#Intermediate intermediate collection points new battery
capi= data.parse("capa", header=None)[1][1]
#Capacity at each workshop for storage
capw= data.parse("capa", header=None)[1][2]
#Capacity transportation CD
captcd = data.parse("capa", header=None)[1][3]
captcdmin = data.parse("capa", header=None)[1][6]
#Capacity manufacturing operation
capm = data.parse("capa", header=None)[1][4]
#Capacity remanufacturing operation
capr = data.parse("capa", header=None)[1][5]

##Variable Costs
#Variable cost of manufacturing new battery
MC= data.parse("var cost", header=None)[1][0]
#Variable cost of remanufacturing battery
RC= data.parse("var cost", header=None)[1][2]
#Variable cost of storage in collection center
UC= data.parse("var cost", header=None)[1][3]
#Variable cost of storage in workshop
WC = data.parse("var cost", header=None)[1][4]
#Variable cost of storage in intermediate collection center
IC = data.parse("var cost", header=None)[1][5]
#Variable cost of Re-X strategy (different reprocessing strategies)
RXC = data.parse("var cost", header=None)[1][6]
#setup cost for manufacturing in period t
MS = data.parse("var cost", header=None)[1][7]
#setup cost for remanufacturing in period t
MR  = data.parse("var cost", header=None)[1][8]
#renting cost transportation C-D
SCTCD = data.parse("var cost", header=None)[1][9]
#handling cost workshop
HW = data.parse("var cost", header=None)[1][10]

##Transportation Costs per battery per km
#Used batteries
TCu = data.parse("transportation cost", header=None)[1][0]
#Damaged batteries
TCd = data.parse("transportation cost", header=None)[1][1]
#New batteries to intermediate
TCni = data.parse("transportation cost", header=None)[1][2]
#New batteries to workshop
TCnw = data.parse("transportation cost", header=None)[1][3]

###Revenue
#New batteries
Pn = data.parse("revenue", header=None)[1][0]
#Remanufactured batteries
Pr = data.parse("revenue", header=None)[1][1]

In [ ]:
# Deterministic demand function new (increase factor for base-model: 0.008)
intial_demand=4800

demand=np.matrix([[1]])
for q in range(59):
    demand=np.r_[demand,[[(1+0.008)**(q+1)]]]
demand=demand*intial_demand/600    
demand=np.ceil(demand)

Dn=np.full([60,601], demand)

# Deterministic demand function remanufactured (increase factor for base-model: 0.025, for sensitivity analysis: +- 20% -> 0.02 and 0.03 / -30% -> 0,0175 / -10% -> 0,0225)
intial_demand=400

demand=np.matrix([[1]])
for q in range(59):
    demand=np.r_[demand,[[(1+0.025)**(q+1)]]]
demand=demand*intial_demand/600    
demand=np.ceil(demand)

Dr=np.full([60,601], demand)

### Definition of decision variables

In [ ]:
## Network opening decision variables are decisions on the first stage (concerns Ac and Bi)
## Second stage decisions include the index s for the different scenarios

## As docplex only allows direct adding of variables up to 3 dimensions, flows have to be indexed like this
Xwc = model.continuous_var_dict((w1, c1, t1, s1)
                                for w1 in w
                                for c1 in c
                                for t1 in t
                                for s1 in s) # old product flowing from w to c in t 
Xwd = model.continuous_var_dict((w2, d2, t2, s2)
                                for w2 in w
                                for d2 in d
                                for t2 in t
                                for s2 in s)# old product flowing from w to d in t 
Xcd = model.continuous_var_dict((c3, d3, t3, s3)
                                for c3 in c
                                for d3 in d
                                for t3 in t
                                for s3 in s)# old product flowing from c to d in t
Ymi = model.continuous_var_dict((m4, i4, t4, s4)
                                for m4 in m
                                for i4 in i
                                for t4 in t
                                for s4 in s)# new product flowing from m to i in t 
Yiw = model.continuous_var_dict((i5, w5, t5, s5)
                                for i5 in i
                                for w5 in w
                                for t5 in t
                                for s5 in s) # new product flowing from i to w in t 

Lin = model.continuous_var_cube(i, t, s, name = "invIn") # inventory level intermediate in t new batteries
Lc = model.continuous_var_cube(c, t, s, name = "invC") # inventory level collection used products in t 
Lnw = model.continuous_var_cube(w, t, s, name = "invWn") # inventory level new batteries at workshop in t 
Luw = model.continuous_var_cube(w, t, s, name = "invWu") # inventory level used batteries at workshop in t 

Ac = model.binary_var_matrix(c, t, name = "setupC")  #Binary for collection center used products
Bi = model.binary_var_matrix(i, t, name = "setupI")  #Binary for intermediate collect. new products
Cw = model.binary_var_cube(w, t, s, name = "setupW")  #Binary for workshop storage
Sm = model.binary_var_cube(m, t, s, name = "setupM")  #Binary for manufacturing setup
Stcd = model.binary_var_cube(c, t, s, name = "setupTranspCD")  #Binary for setup renting transporation C-D

SalesN = model.continuous_var_cube(w, t, s, name = "SalesN") # Sales new batteries at workshop in t 
SalesR = model.continuous_var_cube(w, t, s, name = "SalesR") # Sales remanufactured batteries at workshop in t 

### Definition of constraints

In [ ]:
# satisfy demand in each period - new batteries 
#(demand refers to T-1, as python starts counting from zero, not one// Model meets demand from current period, not from previous period)
for T in t:
    for W in w:
        for S in s:
            model.add_constraint(SalesN[W,T,S]<= Dn[T-1,W-1]) 
        
# satisfy demand in each period - remanufactured batteries -> in baseline model: must be enforced, as they represent a loss (due to this loss, sales will never be larger than Demand) 
for T in t:
    for W in w:
        for S in s:
            model.add_constraint(SalesR[W,T,S]>= Dr[T-1,W-1]) 

# Quality of used batteries
for W in w:  
    for T in range (13,61):
        for S in s:
            model.add_constraint((alpha[W]*beta[W,T,S-1]*SalesN[W,T-12,S]) <= model.sum(Xwd[W,D,T,S] for D in d))

#Capacity constraint used battery collection in collection center C
model.add_constraints(Lc[C,T,S] <= capc*Ac[C,T] for C in c for T in t for S in s)

#Capacity constraint intermediate, new battery collection
model.add_constraints(Lin[I,T,S] <= capi*Bi[I,T] for I in i for T in t  for S in s)

#Capacity constraint workshop for storage
model.add_constraints((Luw[W,T,S]+Lnw[W,T,S]) <= capw*Cw[W,T,S] for W in w for T in t for S in s)

#Capacity manufacturing operation
model.add_constraints(model.sum(Ymi[M,I,T,S] for I in i) <= capm*Sm[M,T, S] for M in m for T in t for S in s)

#Capacity constraint for transportation CD maximum
model.add_constraints(model.sum(Xcd[C,D,T,S] for D in d) <= captcd*Stcd[C,T,S] for T in t for C in c for S in s)

#Capacity constraint for transportation CD minimum for batch transportation
model.add_constraints(model.sum(Xcd[C,D,T,S] for D in d) >= captcdmin*Stcd[C,T,S] for T in t for C in c for S in s)

#Not closing constraint if opened facility (first two constraints are first stage decisions)
model.add_constraints(Ac[C,T]<=Ac[C,T+1] for C in c for T in range (1,60))
model.add_constraints(Bi[I,T]<=Bi[I,T+1] for I in i for T in range (1,60))
model.add_constraints(Cw[W,T,S]<=Cw[W,T+1,S] for S in s for W in w for T in range (1,60))

#Enforce opening of facility (workshops always, collection center and intermediate at least once) (first two constraints are first stage decisions)
model.add_constraint(model.sum(Ac[C,T] for C in c for T in t)>= 1 )
model.add_constraint(model.sum(Bi[I,T] for I in i for T in t)>= 1 )
for W in w:
    for T in t:
        for S in s:
            model.add_constraint(Cw[W,T,S]>= 1)
        
###Inventory Balance for periods 2 to 60
#for collection-center "C"
model.add_constraints(model.sum(Xwc[W,C,T,S] for W in w) + Lc[C,T-1,S] -model.sum(Xcd[C,D,T,S] for D in d) == Lc[C,T,S] for C in c for T in range (2,61) for S in s)

#for intermediate "I"
model.add_constraints(model.sum(Ymi[M,I,T,S] for M in m) + Lin[I,T-1,S] -model.sum(Yiw[I,W,T,S] for W in w) == Lin[I,T,S] for I in i for T in range (2,61) for S in s)
#for Workshops (1. inflow, 2. outflow)
model.add_constraints(model.sum(Yiw[I,W,T,S] for I in i) + Lnw[W,T-1,S] - SalesN[W,T,S] - SalesR[W,T,S] == Lnw[W,T,S]for W in w for T in range (2,61) for S in s)
model.add_constraints(model.sum(alpha[W]*(SalesN[W,T-12,S] + SalesR[W,T-12,S])) + Luw[W,T-1,S] - model.sum(Xwc[W,C,T,S] for C in c)- model.sum(Xwd[W,D,T,S] for D in d) == Luw[W,T,S] for S in s for W in w for T in range (13,61))

#Initial inventory level at period 1 collection point
for C in c:
    for S in s:
        model.add_constraint(Lc[C,1,S]==0)

#Initial inventory level at period 1 intermediate    
for I in i:
    for S in s:
        model.add_constraint(model.sum(Ymi[M,I,1,S] for M in m) + 0 -model.sum(Yiw[I,W,1,S] for W in w) == Lin[I,1,S] )
    
#Initial inventory level at period 1 workshop    
for W in w:
    for S in s:
        model.add_constraint(model.sum(Yiw[I,W,1,S] for I in i) + 10 - SalesN[W,1,S] - SalesR[W,1,S]== Lnw[W,1,S] )
for W in w:
    for T in range (1,13):
        for S in s:
            model.add_constraint(Luw[W,T,S]==0)  

In [ ]:
#Opening collection center
for C in c:
    for T in t:
        for S in s:
            model.add_constraint(model.sum(Xwc[W,C,T,S] for W in w) <= 9999999*Ac[C,T])

#Open max. 4 collection center out of the 4 possible locations (first stage decision)
for T in t:
    model.add_constraint(model.sum(Ac[C,T] for C in c)<=4)

In [ ]:
#Non negativity
model.add_constraints(Xwc[W,C,T,S]>= 0 for W in w for T in t for C in c for S in s)
model.add_constraints(Xwd[W,D,T,S]>= 0 for W in w for T in t for D in d for S in s)
model.add_constraints(Xcd[C,D,T,S]>= 0 for C in c for T in t for D in d for S in s)
model.add_constraints(Ymi[M,I,T,S]>= 0 for M in m for T in t for I in i for S in s)
model.add_constraints(Yiw[I,W,T,S]>= 0 for W in w for T in t for I in i for S in s)

model.add_constraints(Lin[I,T,S]>= 0 for T in t for I in i for S in s)
model.add_constraints(Lc[C,T,S]>= 0 for T in t for C in c for S in s)
model.add_constraints(Luw[W,T,S]>= 0 for T in t for W in w for S in s)
model.add_constraints(Lnw[W,T,S]>= 0 for T in t for W in w for S in s)

### Definition Objective Function

In [ ]:
#Revenue
REV = Pn*model.sum(prob[S-1]*SalesN[W,T,S] for W in w for T in t for S in s) + Pr*model.sum(prob[S-1]*SalesR[W,T,S] for W in w for T in t for S in s)  

In [ ]:
#Fixed opening cost (depreciated, per month) (collection center and intermediate)
FOCC = Fc*model.sum(Ac[C,T] for C in c for T in t)  
FOCI =  Fi*model.sum(Bi[I,T] for I in i for T in t)
#Setup cost manufacturing
SCM = MS*model.sum((prob[S-1]*Sm[M,T,S] for M in m for T in t for S in s))
#Setup cost transportation CD
SCTCDU = SCTCD*model.sum((prob[S-1]*Stcd[C,T,S] for C in c for T in t for S in s))

In [ ]:
## !!!
##Only activate, in case of new sampling for workshops
## !!!

#DF_3=DF_3.sample(n=600)
#DF_3.reset_index(drop=True, inplace=True)
#DF_3.to_csv("workshop_sample.csv")
DF_3=pd.read_csv("workshop_sample.csv")

In [ ]:
#transportation cost new batteries to intermediate
TRCOSTN= TCni*DF_3.loc[1,"dmi"]*(model.sum(prob[S-1]*Ymi[M,I,T,S] for M in m for I in i for T in t for S in s))

In [ ]:
#transportation cost new batteries to workshop
TCnw_matrix=TCnw*DF_3.loc[:,"diw"]
TRCOSTNW= model.sum(prob[S-1]*Yiw[I,W,T,S]*TCnw_matrix[W-1] for I in i for T in t for W in w for S in s)

In [ ]:
#transportation cost used batteries from workshops to C1
TCU_matrix=TCu*DF_3.loc[:,"dwc"]
TRCOSTU1=model.sum(prob[S-1]*Xwc[W,1,T,S]*TCU_matrix[W-1]  for T in t for W in w for S in s)

#transportation cost used batteries from workshops to C2
TCU_matrix2=TCu*(DF_3.loc[:,"dwc2"])
TRCOSTU2=model.sum(prob[S-1]*Xwc[W,2,T,S]*TCU_matrix2[W-1] for T in t for W in w for S in s)

#transportation cost used batteries from workshops to C3
TCU_matrix3=TCu*(DF_3.loc[:,"dwc3"])
TRCOSTU3=model.sum(prob[S-1]*Xwc[W,3,T,S]*TCU_matrix3[W-1] for T in t for W in w for S in s)

#transportation cost used batteries from workshops to C4
TCU_matrix4=TCu*(DF_3.loc[:,"dwc4"])
TRCOSTU4=model.sum(prob[S-1]*Xwc[W,4,T,S]*TCU_matrix4[W-1] for T in t for W in w for S in s)

TRCOSTU =TRCOSTU1 + TRCOSTU2 + TRCOSTU3 + TRCOSTU4

In [ ]:
#transportation cost damaged batteries from workshop
TCd_matrix=TCd*DF_3.loc[:,"dwd"]
TRCOSTWD = model.sum(prob[S-1]*Xwd[W,D,T,S]*TCd_matrix[W-1] for D in d for T in t for W in w for S in s)

In [ ]:
#transportation cost damaged batteries from collection center 1
TRCOSTCD1= TCd*DF_3.loc[1,"dcd"] *model.sum(prob[S-1]*(1-delta[1])*Xcd[1,D,T,S] for D in d for T in t for S in s)

#transportation cost damaged batteries from collection center 2
TRCOSTCD2= TCd*DF_3.loc[1,"dc2d"] *model.sum(prob[S-1]*(1-delta[2])*Xcd[2,D,T,S] for D in d for T in t for S in s)

#transportation cost damaged batteries from collection center 3
TRCOSTCD3= TCd*DF_3.loc[1,"dc3d"] *model.sum(prob[S-1]*(1-delta[3])*Xcd[3,D,T,S] for D in d for T in t for S in s)

#transportation cost damaged batteries from collection center 4
TRCOSTCD4= TCd*DF_3.loc[1,"dc4d"] *model.sum(prob[S-1]*(1-delta[4])*Xcd[4,D,T,S] for D in d for T in t for S in s)

TRCOSTCD=TRCOSTCD1 + TRCOSTCD2 + TRCOSTCD3 + TRCOSTCD4

In [ ]:
#transportation cost used batteries from collection center 1 
TRCOSTCDused1 = TCu*DF_3.loc[1,"dcd"] *model.sum(prob[S-1]*(delta[1])*Xcd[1,D,T,S] for D in d for T in t for S in s)

#transportation cost used batteries from collection center 2 
TRCOSTCDused2 = TCu*DF_3.loc[1,"dc2d"] *model.sum(prob[S-1]*(delta[2])*Xcd[2,D,T,S] for D in d for T in t for S in s)

#transportation cost used batteries from collection center 3 
TRCOSTCDused3 = TCu*DF_3.loc[1,"dc3d"] *model.sum(prob[S-1]*(delta[3])*Xcd[3,D,T,S] for D in d for T in t for S in s)

#transportation cost used batteries from collection center 4 
TRCOSTCDused4 = TCu*DF_3.loc[1,"dc4d"] *model.sum(prob[S-1]*(delta[4])*Xcd[4,D,T,S] for D in d for T in t for S in s)

TRCOSTCDused = TRCOSTCDused1 + TRCOSTCDused2 + TRCOSTCDused3 + TRCOSTCDused4

In [ ]:
#cost of storage in workshop (used batteries outflow)
CWS_U_OUT = WC*(model.sum((prob[S-1]*Luw[W,T,S] for W in w for T in t for S in s)))
#cost of storage in workshop (new batteries inflow)          
CWS_N_IN = WC*(model.sum((prob[S-1]*Lnw[W,T,S] for W in w for T in t for S in s)))

In [ ]:
#Cost for Re-X (1. for damaged batteries directly from W, 2. for damaged batteries from C)
REXC_WD = RXC*(model.sum(prob[S-1]*Xwd[W,D,T,S] for W in w for D in d for T in t for S in s))
REXC_CD = RXC*(model.sum(prob[S-1]*Xcd[C,D,T,S] for C in c for D in d for T in t for S in s))

In [ ]:
#variable cost (manufacturing, intermediate storage, collection center)
VARM = MC*(model.sum(prob[S-1]*Ymi[M,I,T,S] for M in m for I in i for T in t for S in s)) 
VARI_N = IC*(model.sum(prob[S-1]*Lin[I,T,S] for I in i for T in t for S in s))  
VARC = UC*(model.sum(prob[S-1]*Lc[C,T,S] for C in c for T in t for S in s)) 

In [ ]:
#Handling cost in workshop
HanW = HW*(model.sum(prob[S-1]*Xwd[W,D,T,S] for W in w for D in d for T in t for S in s) + model.sum(prob[S-1]*Xwc[W,C,T,S] for W in w for C in c for T in t for S in s))

In [ ]:
#Profit Calculation
P=REV- (HanW + FOCC+FOCI + TRCOSTN + TRCOSTNW + TRCOSTU + TRCOSTCD + TRCOSTCDused + TRCOSTWD + CWS_U_OUT + CWS_N_IN + VARM + VARI_N + VARC + REXC_WD + REXC_CD +SCM + SCTCDU)

In [ ]:
model.maximize(P)
model.solve()

print(model.solve_details)

### Output generation for Excel-Export

In [ ]:
##Definition of new decision variables

YiwNew = model.integer_var_matrix(t, s, name = "agg. flow I-W") #agg. flow I-W"  over W in T
XwcNew = model.integer_var_cube(c, t, s, name = "agg. flow W-C") #agg. flow W-C"  over W in T
XwdNew = model.integer_var_matrix( t, s, name = "agg. flow W-D") #agg. flow W-D"  over D in T
XcdNew = model.integer_var_cube(c, t, s, name = "agg. flow C-D") #agg. flow C-D"  over C in T
YmiNew = model.integer_var_matrix(t, s, name = "agg. flow M-I") #agg. flow M-I"  over I in T
SalesNnew = model.integer_var_matrix(t, s, name = "agg. SalesN") #agg. Sales new batteries over W
SalesRnew = model.integer_var_matrix(t, s,  name = "agg. SalesR") #agg. Sales rem. batteries over W
DemNNew = model.integer_var_dict(t, name = "agg. Demand New") #agg. demand new batteries
DemRNew = model.integer_var_dict(t, name = "agg. Demand Rem") #agg. demand rem batteries

LnwNew = model.integer_var_matrix(t, s, name = "agg. Inw N W") #agg. inventory over all w of new batteries 
LuwNew = model.integer_var_matrix(t, s, name = "agg. Inw U W") #agg. inventory over all w of used batteries
LinNew = model.integer_var_matrix(t, s, name = "agg. Inw N I") #agg. inventory over all i of new batteries
SmNew = model.integer_var_matrix(t, s, name = "agg. setup manu") #setup manufacturing
StcdNew = model.integer_var_cube(c, t, s,  name = "add setup transportation") #agg. transportation setup over all scenarios
CwNew = model.integer_var_matrix(t, s,  name = "agg. setup w") #agg. setup W over all scenarios
                                 

In [ ]:
## Aggregation of results for excel-export

for T in t:
    for S in s:
        YiwNew[T,S] = model.sum(Yiw[I,W,T,S] for W in w for I in i)
    
for T in t:
    for S in s:
        YmiNew[T,S] = model.sum(Ymi[M,I,T,S] for M in m for I in i)       

for T in t:
    for C in c:
        for S in s:
            XwcNew[C,T,S] = model.sum(Xwc[W,C,T,S] for W in w)

for T in t:
    for S in s:
        XwdNew[T,S] = model.sum(Xwd[W,D,T,S] for D in d for W in w)
    
for T in t:
    for C in c:
        for S in s:
            XcdNew[C,T,S] = model.sum(Xcd[C,D,T,S] for D in d)

for T in t:
    for S in s:
        LnwNew[T,S] = model.sum(Lnw[W,T,S] for W in w)
for T in t:
    for S in s:
        LuwNew[T,S] = model.sum(Luw[W,T,S] for W in w)
    
for T in t:
    for S in s:
        LinNew[T,S] = model.sum(Lin[I,T,S] for I in i)    
    
for T in t:
    for S in s:
        SalesNnew[T,S] = model.sum(SalesN[W,T,S] for W in w)
for T in t:
    for S in s:
        SalesRnew[T,S] = model.sum(SalesR[W,T,S] for W in w)

for T in t:
    for S in s:
        SmNew[T,S] = model.sum(Sm[M,T,S] for M in m)

for T in t:
    for S in s:
        CwNew[T,S] = model.sum(Cw[W,T,S] for W in w)
        
        
### Dataframe for scenario 1

header = ['Period', 'Flow M-I' , 'Flow I-W' , 'Flow W-C1', 'Flow W-C2', 'Flow W-C3', 'Flow W-C4', 'Flow W-D', 'Flow C-D1', 'Flow C-D2', 'Flow C-D3', 'Flow C-D4', 'InvIn', 'InvC1', 'InvC2', 'InvC3', 'InvC4', 'InvWn', 'InvWu', 'SetupM',  'SetupTransportCD1',  'SetupTransportCD2',  'SetupTransportCD3',  'SetupTransportCD4', 'setupC1', 'setupC2', 'setupC3', 'setupC4', 'setupI' ,'setupW', 'SalesN', 'SalesR']
df = pd.DataFrame(columns = header)
for T in t:
    df.loc[len(df), 'Period'] = T
    df.loc[len(df)-1, 'Flow M-I'] = YmiNew[T,1].solution_value
    df.loc[len(df)-1, 'Flow I-W'] = YiwNew[T,1].solution_value
    df.loc[len(df)-1, 'Flow W-C1'] = XwcNew[1,T,1].solution_value
    df.loc[len(df)-1, 'Flow W-C2'] = XwcNew[2,T,1].solution_value
    df.loc[len(df)-1, 'Flow W-C3'] = XwcNew[3,T,1].solution_value
    df.loc[len(df)-1, 'Flow W-C4'] = XwcNew[4,T,1].solution_value
    df.loc[len(df)-1, 'Flow W-D'] = XwdNew[T,1].solution_value
    df.loc[len(df)-1, 'Flow C-D1'] = XcdNew[1,T,1].solution_value
    df.loc[len(df)-1, 'Flow C-D2'] = XcdNew[2,T,1].solution_value
    df.loc[len(df)-1, 'Flow C-D3'] = XcdNew[3,T,1].solution_value
    df.loc[len(df)-1, 'Flow C-D4'] = XcdNew[4,T,1].solution_value
    df.loc[len(df)-1, 'InvIn'] = LinNew[T,1].solution_value
    df.loc[len(df)-1, 'InvWn'] = LnwNew[T,1].solution_value
    df.loc[len(df)-1, 'InvWu'] = LuwNew[T,1].solution_value
    df.loc[len(df)-1, 'InvC1'] = Lc[1,T,1].solution_value
    df.loc[len(df)-1, 'InvC2'] = Lc[2,T,1].solution_value
    df.loc[len(df)-1, 'InvC3'] = Lc[3,T,1].solution_value
    df.loc[len(df)-1, 'InvC4'] = Lc[4,T,1].solution_value
    df.loc[len(df)-1, 'SetupM'] = SmNew[T,1].solution_value
    df.loc[len(df)-1, 'SetupTransportCD1'] = Stcd[1,T,1].solution_value
    df.loc[len(df)-1, 'SetupTransportCD2'] = Stcd[2,T,1].solution_value
    df.loc[len(df)-1, 'SetupTransportCD3'] = Stcd[3,T,1].solution_value
    df.loc[len(df)-1, 'SetupTransportCD4'] = Stcd[4,T,1].solution_value
    df.loc[len(df)-1, 'setupC1'] = Ac[1,T].solution_value
    df.loc[len(df)-1, 'setupC2'] = Ac[2,T].solution_value
    df.loc[len(df)-1, 'setupC3'] = Ac[3,T].solution_value
    df.loc[len(df)-1, 'setupC4'] = Ac[4,T].solution_value
    df.loc[len(df)-1, 'setupI'] = Bi[I,T].solution_value
    df.loc[len(df)-1, 'setupW'] = CwNew[T,1].solution_value
    df.loc[len(df)-1, 'SalesN'] = SalesNnew[T,1].solution_value
    df.loc[len(df)-1, 'SalesR'] = SalesRnew[T,1].solution_value
print(df)
print("The objective value is ", model.objective_value)
print('REV value is:', REV.solution_value)
print('FOCC value is:', FOCC.solution_value)
print('FOCI value is:', FOCI.solution_value)
print('TRCOSTN value is:', TRCOSTN.solution_value)
print('TRCOSTNW value is:', TRCOSTNW.solution_value)
print('TRCOSTU value is:', TRCOSTU.solution_value)
print('TRCOSTCD value is:', TRCOSTCD.solution_value)
print('TRCOSTWD value is:', TRCOSTWD.solution_value)
print('TRCOSTCDused value is:', TRCOSTCDused.solution_value)
print('CWS_N_IN value is:', CWS_N_IN.solution_value)
print('CWS_U_OUT value is:', CWS_U_OUT.solution_value)
print('VARM value is:', VARM.solution_value)
print('VARI_N value is:', VARI_N.solution_value)
print('VARC value is:', VARC.solution_value)
print('REXC_WD value is:', REXC_WD.solution_value)
print('REXC_CD value is:', REXC_CD.solution_value)
print('SCM value is:', SCM.solution_value)
print('SCTCDU value is:', SCTCDU.solution_value)
print('HanW value is:', HanW.solution_value)

In [ ]:
### Dataframe for scenario 2

header = ['Period', 'Flow M-I' , 'Flow I-W' , 'Flow W-C1', 'Flow W-C2', 'Flow W-C3', 'Flow W-C4', 'Flow W-D', 'Flow C-D1', 'Flow C-D2', 'Flow C-D3', 'Flow C-D4', 'InvIn', 'InvC1', 'InvC2', 'InvC3', 'InvC4', 'InvWn', 'InvWu', 'SetupM',  'SetupTransportCD1',  'SetupTransportCD2',  'SetupTransportCD3',  'SetupTransportCD4', 'setupC1', 'setupC2', 'setupC3', 'setupC4', 'setupI' ,'setupW', 'SalesN', 'SalesR']
df2 = pd.DataFrame(columns = header)
for T in t:
    df2.loc[len(df2), 'Period'] = T
    df2.loc[len(df2)-1, 'Flow M-I'] = YmiNew[T,2].solution_value
    df2.loc[len(df2)-1, 'Flow I-W'] = YiwNew[T,2].solution_value
    df2.loc[len(df2)-1, 'Flow W-C1'] = XwcNew[1,T,2].solution_value
    df2.loc[len(df2)-1, 'Flow W-C2'] = XwcNew[2,T,2].solution_value
    df2.loc[len(df2)-1, 'Flow W-C3'] = XwcNew[3,T,2].solution_value
    df2.loc[len(df2)-1, 'Flow W-C4'] = XwcNew[4,T,2].solution_value
    df2.loc[len(df2)-1, 'Flow W-D'] = XwdNew[T,2].solution_value
    df2.loc[len(df2)-1, 'Flow C-D1'] = XcdNew[1,T,2].solution_value
    df2.loc[len(df2)-1, 'Flow C-D2'] = XcdNew[2,T,2].solution_value
    df2.loc[len(df2)-1, 'Flow C-D3'] = XcdNew[3,T,2].solution_value
    df2.loc[len(df2)-1, 'Flow C-D4'] = XcdNew[4,T,2].solution_value
    df2.loc[len(df2)-1, 'InvIn'] = LinNew[T,2].solution_value
    df2.loc[len(df2)-1, 'InvWn'] = LnwNew[T,2].solution_value
    df2.loc[len(df2)-1, 'InvWu'] = LuwNew[T,2].solution_value
    df2.loc[len(df2)-1, 'InvC1'] = Lc[1,T,2].solution_value
    df2.loc[len(df2)-1, 'InvC2'] = Lc[2,T,2].solution_value
    df2.loc[len(df2)-1, 'InvC3'] = Lc[3,T,2].solution_value
    df2.loc[len(df2)-1, 'InvC4'] = Lc[4,T,2].solution_value
    df2.loc[len(df2)-1, 'SetupM'] = SmNew[T,2].solution_value
    df2.loc[len(df2)-1, 'SetupTransportCD1'] = Stcd[1,T,2].solution_value
    df2.loc[len(df2)-1, 'SetupTransportCD2'] = Stcd[2,T,2].solution_value
    df2.loc[len(df2)-1, 'SetupTransportCD3'] = Stcd[3,T,2].solution_value
    df2.loc[len(df2)-1, 'SetupTransportCD4'] = Stcd[4,T,2].solution_value
    df2.loc[len(df2)-1, 'setupC1'] = Ac[1,T].solution_value
    df2.loc[len(df2)-1, 'setupC2'] = Ac[2,T].solution_value
    df2.loc[len(df2)-1, 'setupC3'] = Ac[3,T].solution_value
    df2.loc[len(df2)-1, 'setupC4'] = Ac[4,T].solution_value
    df2.loc[len(df2)-1, 'setupI'] = Bi[I,T].solution_value
    df2.loc[len(df2)-1, 'setupW'] = CwNew[T,2].solution_value
    df2.loc[len(df2)-1, 'SalesN'] = SalesNnew[T,2].solution_value
    df2.loc[len(df2)-1, 'SalesR'] = SalesRnew[T,2].solution_value
print(df2)


In [ ]:
### Dataframe for scenario 3

header = ['Period', 'Flow M-I' , 'Flow I-W' , 'Flow W-C1', 'Flow W-C2', 'Flow W-C3', 'Flow W-C4', 'Flow W-D', 'Flow C-D1', 'Flow C-D2', 'Flow C-D3', 'Flow C-D4', 'InvIn', 'InvC1', 'InvC2', 'InvC3', 'InvC4', 'InvWn', 'InvWu', 'SetupM',  'SetupTransportCD1',  'SetupTransportCD2',  'SetupTransportCD3',  'SetupTransportCD4', 'setupC1', 'setupC2', 'setupC3', 'setupC4', 'setupI' ,'setupW', 'SalesN', 'SalesR']
df3 = pd.DataFrame(columns = header)
for T in t:
    df3.loc[len(df3), 'Period'] = T
    df3.loc[len(df3)-1, 'Flow M-I'] = YmiNew[T,3].solution_value
    df3.loc[len(df3)-1, 'Flow I-W'] = YiwNew[T,3].solution_value
    df3.loc[len(df3)-1, 'Flow W-C1'] = XwcNew[1,T,3].solution_value
    df3.loc[len(df3)-1, 'Flow W-C2'] = XwcNew[2,T,3].solution_value
    df3.loc[len(df3)-1, 'Flow W-C3'] = XwcNew[3,T,3].solution_value
    df3.loc[len(df3)-1, 'Flow W-C4'] = XwcNew[4,T,3].solution_value
    df3.loc[len(df3)-1, 'Flow W-D'] = XwdNew[T,3].solution_value
    df3.loc[len(df3)-1, 'Flow C-D1'] = XcdNew[1,T,3].solution_value
    df3.loc[len(df3)-1, 'Flow C-D2'] = XcdNew[2,T,3].solution_value
    df3.loc[len(df3)-1, 'Flow C-D3'] = XcdNew[3,T,3].solution_value
    df3.loc[len(df3)-1, 'Flow C-D4'] = XcdNew[4,T,3].solution_value
    df3.loc[len(df3)-1, 'InvIn'] = LinNew[T,3].solution_value
    df3.loc[len(df3)-1, 'InvWn'] = LnwNew[T,3].solution_value
    df3.loc[len(df3)-1, 'InvWu'] = LuwNew[T,3].solution_value
    df3.loc[len(df3)-1, 'InvC1'] = Lc[1,T,3].solution_value
    df3.loc[len(df3)-1, 'InvC2'] = Lc[2,T,3].solution_value
    df3.loc[len(df3)-1, 'InvC3'] = Lc[3,T,3].solution_value
    df3.loc[len(df3)-1, 'InvC4'] = Lc[4,T,3].solution_value
    df3.loc[len(df3)-1, 'SetupM'] = SmNew[T,3].solution_value
    df3.loc[len(df3)-1, 'SetupTransportCD1'] = Stcd[1,T,3].solution_value
    df3.loc[len(df3)-1, 'SetupTransportCD2'] = Stcd[2,T,3].solution_value
    df3.loc[len(df3)-1, 'SetupTransportCD3'] = Stcd[3,T,3].solution_value
    df3.loc[len(df3)-1, 'SetupTransportCD4'] = Stcd[4,T,3].solution_value
    df3.loc[len(df3)-1, 'setupC1'] = Ac[1,T].solution_value
    df3.loc[len(df3)-1, 'setupC2'] = Ac[2,T].solution_value
    df3.loc[len(df3)-1, 'setupC3'] = Ac[3,T].solution_value
    df3.loc[len(df3)-1, 'setupC4'] = Ac[4,T].solution_value
    df3.loc[len(df3)-1, 'setupI'] = Bi[I,T].solution_value
    df3.loc[len(df3)-1, 'setupW'] = CwNew[T,3].solution_value
    df3.loc[len(df3)-1, 'SalesN'] = SalesNnew[T,3].solution_value
    df3.loc[len(df3)-1, 'SalesR'] = SalesRnew[T,3].solution_value
print(df3)


In [ ]:
##Dataframe for profit and costs results

header = ['Objective value','REV' , 'FOCC', 'FOCI','TRCOSTN',  'TRCOSTNW', 'TRCOSTU', 'TRCOSTCD', 'TRCOSTWD','TRCOSTCDused','CWS_N_IN' ,'CWS_U_OUT', 'VARM', 'VARI_N', 'VARC' ,  'REXC_CD' ,  'REXC_WD', 'SCM', 'SCTCDU', 'HanW']
df4 = pd.DataFrame(columns = header)

df4.loc[len(df)-1, 'Objective value'] = model.objective_value
df4.loc[len(df)-1, 'FOCC'] = FOCC.solution_value
df4.loc[len(df)-1, 'FOCI'] = FOCI.solution_value
df4.loc[len(df)-1, 'REV'] = REV.solution_value
df4.loc[len(df)-1, 'TRCOSTN'] = TRCOSTN.solution_value
df4.loc[len(df)-1, 'TRCOSTNW'] = TRCOSTNW.solution_value
df4.loc[len(df)-1, 'TRCOSTU'] = TRCOSTU.solution_value
df4.loc[len(df)-1, 'TRCOSTCD'] = TRCOSTCD.solution_value
df4.loc[len(df)-1, 'TRCOSTWD'] = TRCOSTWD.solution_value
df4.loc[len(df)-1, 'TRCOSTCDused'] = TRCOSTCDused.solution_value
df4.loc[len(df)-1, 'CWS_N_IN'] = CWS_N_IN.solution_value
df4.loc[len(df)-1, 'CWS_U_OUT'] = CWS_U_OUT.solution_value
df4.loc[len(df)-1, 'VARM'] = VARM.solution_value
df4.loc[len(df)-1, 'VARI_N'] = VARI_N.solution_value
df4.loc[len(df)-1, 'VARC'] = VARC.solution_value
df4.loc[len(df)-1, 'REXC_CD'] = REXC_CD.solution_value
df4.loc[len(df)-1, 'REXC_WD'] = REXC_WD.solution_value
df4.loc[len(df)-1, 'SCM'] = SCM.solution_value
df4.loc[len(df)-1, 'SCTCDU'] = SCTCDU.solution_value
df4.loc[len(df)-1, 'HanW'] = HanW.solution_value

In [ ]:
##Dataframe for demand per period

for T in range (0,61):
    DemNNew[T-1] = model.sum(Dn[T-1,W-1] for W in w)
for T in range (0,61):
    DemRNew[T-1] = model.sum(Dr[T-1,W-1] for W in w)
    
header = [ 'DemNAgg', 'DemRAgg']
df5 = pd.DataFrame(columns = header)
for T in range (0,61):
        df5.loc[len(df5), 'Period'] = T
        df5.loc[len(df5)-1, 'DemNAgg'] = DemNNew[T].solution_value
        df5.loc[len(df5)-1, 'DemRAgg'] = DemRNew[T].solution_value 

In [ ]:
##Excel export

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Solution_Baseline.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
df.to_excel(writer, sheet_name='Results_pessim')
df2.to_excel(writer, sheet_name='Results_normal')
df3.to_excel(writer, sheet_name='Results_optimist')
df4.to_excel(writer, sheet_name='Sum values')
df5.to_excel(writer, sheet_name="Demand")
# Close the Pandas Excel writer and output the Excel file.
writer.save()